In [1]:
import numpy as np

class ImageBuffer(object):

    # idx starts from 1. The first image will have idx 1.
    # When the buffer get full with batch size 100, then there will be 100 iamges and the current idx will be 100.

    def __init__(self, height, width, stepsize, channel_size, buffer_size):
        self.height = height
        self.width = width
        self.stepsize = stepsize
        self.channel_size = channel_size
        self.step_channelsize = stepsize * channel_size

        self.buffer = []
        self.buffer_size = buffer_size
        self.count = 0

        assert self.stepsize <= self.buffer_size

        self.start_adjust = (self.stepsize - 1)
        self.full_count = 0

        for _ in range(stepsize):
            self.buffer.append(np.zeros([channel_size,height,width]))


    # [height, width, channel]
    def dm_add(self, frame):
        frame = np.moveaxis(frame, [0, 1, 2], [1, 2, 0])
        if self.count < self.buffer_size:
            self.count += 1
            self.buffer.append(frame)
        else:
            self.buffer.pop(0)
            self.buffer.append(frame)
            self.full_count += 1

    def get_state(self, idx):
        assert idx > 0 and idx <= self.count + self.full_count

        if self.count < self.buffer_size:
            temp_idx = idx + self.start_adjust
        else:
            temp_idx = idx + self.start_adjust - self.full_count

        assert temp_idx > self.start_adjust

        return_array = np.concatenate(self.buffer[temp_idx - self.start_adjust: temp_idx + 1], axis=0)

        print(return_array.shape)

        assert return_array.shape == (self.step_channelsize, self.height, self.width)

        return return_array
        #[stepsize, height, width]

    def get_state_and_next(self, idx):
        assert idx > 0 and idx < self.count + self.full_count
        return self.get_state(idx), self.get_state(idx+1)

In [2]:
base = np.ones([48,64,3])

In [3]:
a = ImageBuffer(48, 64, 10, 3, 10)

In [4]:
for i in range(1,11):
    a.dm_add(base*i)

In [5]:
a.dm_add(base*11)

In [6]:
a.get_state(1)[-6]

AssertionError: 

In [ ]:
buffer_size = 2e5

In [ ]:
buffer_size*1.01

In [ ]:
for i in range(1,8):
    a.dm_add(i)

In [ ]:
a.get_state(7)

In [ ]:
a.dm_add(1)

print(a.get_state(1))

print (a.count)
print (a.full_count)

In [ ]:
a.dm_add(2)

print(a.get_state(2))

print (a.count)
print (a.full_count)

In [ ]:
a.dm_add(3)

print(a.get_state(3))

print (a.count)
print (a.full_count)

In [ ]:
a.dm_add(4)

print(a.get_state(4))

print (a.count)
print (a.full_count)

In [ ]:
a.dm_add(5)

print(a.get_state(5))

print (a.count)
print (a.full_count)

In [ ]:
a.dm_add(6)

print(a.get_state(6))

print (a.count)
print (a.full_count)

In [ ]:
print(a.get_state(6))

In [ ]:
a.buffer

In [ ]:
from dm_control import suite
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
env = suite.load(domain_name="cartpole",task_name="swingup")

In [ ]:
env.reset()

In [ ]:
frame = env.physics.render(camera_id=0, width=640, height=480)

frame.shape

np.moveaxis(frame,[0,1,2],[1,2,0]).shape

plt.imshow(frame)

In [ ]:
env.step(1.0)

frame = env.physics.render(camera_id=0, width=640, height=480)

frame.shape

np.moveaxis(frame,[0,1,2],[1,2,0]).shape

plt.imshow(frame)

In [ ]:
env.step(1.0)

frame = env.physics.render(camera_id=0, width=640, height=480)

frame.shape

np.moveaxis(frame,[0,1,2],[1,2,0]).shape

plt.imshow(frame)

In [ ]:
for _ in range(10):
    env.step(1.0)

    frame = env.physics.render(camera_id=0, width=64, height=48)

    frame.shape

    np.moveaxis(frame,[0,1,2],[1,2,0]).shape

plt.imshow(frame)

In [ ]:
env.step(1.0)

frame = env.physics.render(camera_id=0, width=640, height=480)

frame.shape

np.moveaxis(frame,[0,1,2],[1,2,0]).shape

plt.imshow(frame)

In [ ]:
env.step(1.0)

frame = env.physics.render(camera_id=0, width=640, height=480)

frame.shape

np.moveaxis(frame,[0,1,2],[1,2,0]).shape

plt.imshow(frame)

In [ ]:
plt.imshow(frame)

In [ ]:
conv1 = torch.nn.Conv2d(in_channels = 3, out_channels= 256,
                                     kernel_size= 3, stride= 2, padding= 1)
conv2 = torch.nn.Conv2d(in_channels=256, out_channels=128,
                                     kernel_size=3, stride=2, padding=1)
conv3 = torch.nn.Conv2d(in_channels=128, out_channels=64,
                                     kernel_size=3, stride=2, padding=1)
conv4 = torch.nn.Conv2d(in_channels=64, out_channels=32,
                                     kernel_size=3, stride=2, padding=1)

In [ ]:
frame = frame / 255 - 0.5

In [ ]:
cv1 = conv1(torch.FloatTensor(frame).view([1,3,48,64]))

In [ ]:
cv1.shape

In [ ]:
cv2 = conv2(cv1)

cv2.shape

In [ ]:
cv3 = conv3(cv2)

cv3.shape

In [ ]:
cv4 = conv4(cv3)

cv4.shape

In [ ]:
fc1 = cv4.view([1,-1])
fc1.shape

In [ ]:
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal

class VanillaSACPolicy(nn.Module):
    def __init__(self, state_dim, action_dim, lr, device):
        super(VanillaSACPolicy, self).__init__()

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.actor_lr = lr
        self.device = device


        self.fc1 = nn.Linear(state_dim, 256).to(device)
        self.fc2 = nn.Linear(256, 256).to(device)
        self.mu = nn.Linear(256, action_dim).to(device)
        self.log_std = nn.Linear(256, action_dim).to(device)

        nn.init.uniform_(tensor=self.mu.weight, a = -3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.mu.bias, a=-3e-3, b=3e-3)

        nn.init.uniform_(tensor=self.log_std.weight, a=-3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.log_std.bias, a=-3e-3, b=3e-3)

        self.optimizer = optim.Adam(self.parameters(),lr)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        mu = F.relu(self.mu(x))
        log_std = F.relu(self.log_std(x))
        

        # It should return mu, log_std

        return mu, log_std

    def sample_with_logp(self, x):
        mu, log_std = self.forward(x)
        std = torch.exp(log_std)

        normal = Normal(mu, std)
        x_t = normal.rsample()
        logp = normal.log_prob(x_t)

        y_t = torch.tanh(x_t)
        logp -= torch.log(1 - torch.pow(y_t, 2) + 1e-6)
        
        return y_t, logp
    
    def sample(self, x):
        mu, log_std = self.forward(x)
        std = torch.exp(log_std)
        
        normal = Normal(mu, std)
        
        return normal.sample()

In [ ]:
x,y = torch.FloatTensor([[1,2],[3,4]])

In [ ]:
x

In [ ]:
y

In [ ]:
t = VanillaSACPolicy(1,1,1e-4,torch.device("cuda"))

In [ ]:
mu, log_std = t.forward(torch.FloatTensor([0.0]).to(torch.device("cuda")))

In [ ]:
k = t.sample_with_logp(torch.FloatTensor([0.0]).to(torch.device("cuda")))

In [ ]:
k

In [ ]:
kk = torch.sum(k)

In [ ]:
kk.backward()

In [ ]:
t.fc3.weight.grad[0][:20]

In [ ]:
with torch.no_grad():
    a = t.forward(torch.FloatTensor([1.0]).to(torch.device("cuda")))
    aa = torch.mean(a)
    aa.backward()

In [ ]:
a = t.forward(torch.FloatTensor([1.0]).to(torch.device("cuda")))
aa = torch.mean(a)
aa.backward()

In [ ]:
t.fc3.weight.grad[0][:20]

In [ ]:
a = t.forward(torch.FloatTensor([1.0]).to(torch.device("cuda")))
aa = torch.mean(a, dim=2,keepdim=True)

In [ ]:
aa.shape

In [ ]:
a.shape

In [ ]:
aa.grad_fn

In [ ]:
k = aa.detach()

In [ ]:
k.grad_fn

In [ ]:
torch.min(torch.rand([128,2]),dim=1,keepdim=True)[0]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class VanillaSACQNet(nn.Module):
    def __init__(self, state_dim, action_dim, lr, device):
        super(VanillaSACQNet, self).__init__()

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.actor_lr = lr
        self.device = device

        # Q1 architecture
        self.fc1 = nn.Linear(state_dim + action_dim, 256).to(device)
        self.fc2 = nn.Linear(256,256).to(device)
        self.fc3 = nn.Linear(256,1).to(device)

        nn.init.uniform_(tensor=self.fc3.weight, a = -3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.fc3.bias, a=-3e-3, b=3e-3)

        # Q2 architecture
        self.fc4 = nn.Linear(state_dim + action_dim, 256).to(device)
        self.fc5 = nn.Linear(256, 256).to(device)
        self.fc6 = nn.Linear(256, 1).to(device)

        nn.init.uniform_(tensor=self.fc6.weight, a=-3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.fc6.bias, a=-3e-3, b=3e-3)

        self.optimizer = optim.Adam(self.parameters(),lr)


    def forward(self, s, a):
        x = torch.cat([s,a],dim=1) # [batch, s+a]

        x1 = F.relu(self.fc1(x))
        x1 = F.relu(self.fc2(x1))
        x1 = self.fc3(x1)

        x2 = F.relu(self.fc4(x))
        x2 = F.relu(self.fc5(x2))
        x2 = self.fc6(x2)

        return x1, x2

class ConvSACQNet(nn.Module):
    def __init__(self, step_channelsize, height, width, action_dim, lr, device):
        super(ConvSACQNet, self).__init__()

        self.step_channelsize = step_channelsize
        self.height = height
        self.width = width
        self.out_channels = 16

        self.conv_flatten_size = int(height * width * self.out_channels / (16 ** 2))

        self.action_dim = action_dim
        self.actor_lr = lr
        self.device = device

        # Q1 architecture
        self.conv1 = torch.nn.Conv2d(in_channels=self.step_channelsize, out_channels=64,
                                kernel_size=7, stride=4, padding=3).to(device)
        self.conv2 = torch.nn.Conv2d(in_channels=64, out_channels=self.out_channels,
                                kernel_size=7, stride=4, padding=3).to(device)

        self.fc1 = nn.Linear(self.conv_flatten_size + self.action_dim, 128).to(device)
        self.fc2 = nn.Linear(128, 1).to(device)

        nn.init.uniform_(tensor=self.fc2.weight, a = -3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.fc2.bias, a=-3e-3, b=3e-3)

        # Q2 architecture
        self.conv3 = torch.nn.Conv2d(in_channels=self.step_channelsize, out_channels=64,
                                     kernel_size=7, stride=4, padding=3).to(device)
        self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=self.out_channels,
                                     kernel_size=7, stride=4, padding=3).to(device)

        self.fc3 = nn.Linear(self.conv_flatten_size + self.action_dim, 128).to(device)
        self.fc4 = nn.Linear(128, 1).to(device)

        nn.init.uniform_(tensor=self.fc4.weight, a=-3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.fc4.bias, a=-3e-3, b=3e-3)

        self.optimizer = optim.Adam(self.parameters(),lr)

    def forward(self, s, a):

        x1 = F.relu(self.conv1(s))
        x1 = F.relu(self.conv2(x1))

        x1 = x1.view(-1, self.conv_flatten_size)
        x1 = torch.cat([x1, a], dim=1)  # [batch, s+a]

        x1 = F.relu(self.fc1(x1))
        x1 = self.fc2(x1)

        x2 = F.relu(self.conv3(s))
        x2 = F.relu(self.conv4(x2))

        x2 = x2.view(-1, self.conv_flatten_size)
        x2 = torch.cat([x2, a], dim=1)  # [batch, s+a]

        x2 = F.relu(self.fc3(x2))
        x2 = self.fc4(x2)

        return x1, x2

In [ ]:
QNet = ConvSACQNet(15, 48, 64,1, 1e-3,torch.device("cuda"))

In [ ]:
QNet.state_dict().keys()

In [ ]:
QNet.optimizer.state_dict()

In [ ]:
def torch_network_save(net, path):
    optimizer = net.optimizer.state_dict()
    parameters = net.state_dict()

    torch.save(
        {
            'model_state_dict' : parameters,
            'optimizer_state_dict' : optimizer
        }, path)

def torch_network_load(net, path, device=torch.device("cuda")):
    load_dict = torch.load(path,device)
    parameters = load_dict["model_state_dict"]
    optimizer = load_dict["optimizer_state_dict"]

    net.load_state_dict(parameters)
    net.optimizer.load_state_dict(optimizer)

In [ ]:
torch_network_save(QNet,"./test.torch")

In [7]:
QNet.fc2.weight[0][:10]

NameError: name 'QNet' is not defined

In [ ]:
QNet2 = ConvSACQNet(15, 48, 64,1, 1e-3,torch.device("cuda"))

In [ ]:
QNet2.fc2.weight[0][:10]

In [ ]:
torch_network_load(QNet2,"./test.torch")

In [ ]:
QNet2.fc2.weight[0][:10]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal

class VanillaSACPolicy(nn.Module):
    def __init__(self, state_dim, action_dim, lr, device):
        super(VanillaSACPolicy, self).__init__()

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.actor_lr = lr
        self.device = device


        self.fc1 = nn.Linear(state_dim, 256).to(device)
        self.fc2 = nn.Linear(256, 256).to(device)
        self.mu = nn.Linear(256, action_dim).to(device)
        self.log_std = nn.Linear(256, action_dim).to(device)

        nn.init.uniform_(tensor=self.mu.weight, a = -3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.mu.bias, a=-3e-3, b=3e-3)

        nn.init.uniform_(tensor=self.log_std.weight, a=-3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.log_std.bias, a=-3e-3, b=3e-3)

        self.optimizer = optim.Adam(self.parameters(),lr)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        mu = self.mu(x)
        log_std = self.log_std(x)

        # It should return mu, log_std

        return mu, log_std

    def sample_with_logp(self, x):
        mu, log_std = self.forward(x)
        std = torch.exp(log_std)

        normal = Normal(mu, std)
        x_t = normal.rsample()
        logp = normal.log_prob(x_t)

        y_t = torch.tanh(x_t)
        logp -= torch.log(1 - torch.pow(y_t, 2) + 1e-6)

        return y_t, logp

    def sample(self, x):

        with torch.no_grad():
            mu, log_std = self.forward(x)
            std = torch.exp(log_std)

            normal = Normal(mu, std)

            return torch.tanh(normal.sample())

    def mean_action(self, x):
        with torch.no_grad():
            mu, _log_std = self.forward(x)
            return torch.tanh(mu)






class ConvSACPolicy(nn.Module):
    def __init__(self, step_channelsize, height, width, action_dim, lr, device):
        super(ConvSACPolicy, self).__init__()

        self.step_channelsize = step_channelsize
        self.height = height
        self.width = width
        self.out_channels = 16

        self.conv_flatten_size = int(height * width * self.out_channels / (16 ** 2))

        self.action_dim = action_dim
        self.actor_lr = lr
        self.device = device

        self.conv1 = torch.nn.Conv2d(in_channels=self.step_channelsize, out_channels=64,
                                kernel_size=7, stride=4, padding=3).to(device)
        self.conv2 = torch.nn.Conv2d(in_channels=64, out_channels=self.out_channels,
                                kernel_size=7, stride=4, padding=3).to(device)

        self.mu = nn.Linear(self.conv_flatten_size, action_dim).to(device)
        self.log_std = nn.Linear(self.conv_flatten_size, action_dim).to(device)

        nn.init.uniform_(tensor=self.mu.weight, a = -3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.mu.bias, a=-3e-3, b=3e-3)

        nn.init.uniform_(tensor=self.log_std.weight, a=-3e-3, b=3e-3)
        nn.init.uniform_(tensor=self.log_std.bias, a=-3e-3, b=3e-3)

        self.optimizer = optim.Adam(self.parameters(),lr)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        x = x.view(-1,self.conv_flatten_size)
        print(x)

        mu = self.mu(x)
        log_std = self.log_std(x)

        # It should return mu, log_std

        return mu, log_std

    def sample_with_logp(self, x):
        mu, log_std = self.forward(x)
        std = torch.exp(log_std)

        normal = Normal(mu, std)
        x_t = normal.rsample()
        logp = normal.log_prob(x_t)

        y_t = torch.tanh(x_t)
        logp -= torch.log(1 - torch.pow(y_t, 2) + 1e-6)

        return y_t, logp

    def sample(self, x):

        with torch.no_grad():
            mu, log_std = self.forward(x)
            std = torch.exp(log_std)

            normal = Normal(mu, std)

            return torch.tanh(normal.sample())

    def mean_action(self, x):
        with torch.no_grad():
            mu, _log_std = self.forward(x)
            return torch.tanh(mu)




In [ ]:
F.celu

In [ ]:
stepsize = 10
channelsize = 3

step_channelsize = stepsize * channelsize

height = 48
width = 64

action_dim = env.action_spec().shape[-1]

lr = 1e-3
device = torch.device("cuda")

In [ ]:
policy = ConvSACPolicy(step_channelsize, height, width, action_dim, lr, device)

In [ ]:
torch_network_load(policy,"trained/ConvSACPI-DEBUG_1.torch")

In [ ]:
env.reset()

In [ ]:
frame = env.physics.render(camera_id=0, width=64, height=48)
frame = np.moveaxis(frame,[0,1,2],[1,2,0])

In [ ]:
tt = np.concatenate([np.zeros([27,48,64]),frame],axis=0)

In [ ]:
tt.shape

In [ ]:
plt.imshow(np.moveaxis(frame,[1,2,0],[0,1,2]))

In [ ]:
ttt= np.random.randn(30,48,64)

In [ ]:
ttt.shape

In [ ]:
policy.forward(torch.FloatTensor(ttt).to(device).view(-1,step_channelsize,height,width))

In [8]:
64 * 48 / 16

192.0